# Kombinieren und Zusammenführen von Datensätzen

Die in pandas-Objekten enthaltenen Daten können auf verschiedene Weise miteinander kombiniert werden:

* [pandas.merge](https://pandas.pydata.org/docs/reference/api/pandas.merge.html) verbindet Zeilen in DataFrames basierend auf einem oder mehreren Schlüsseln. Diese Funktion ist von SQL oder anderen relationalen Datenbanken vertraut, da sie Datenbank-Join-Operationen implementiert.
* [pandas.concat](https://pandas.pydata.org/docs/reference/api/pandas.concat.html) verkettet oder *stapelt* Objekte entlang einer Achse.
* Die Instanzmethoden [pandas.DataFrame.combine_first](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.combine_first.html) oder [pandas.Series.combine_first](https://pandas.pydata.org/docs/reference/api/pandas.Series.combine_first.html) ermöglichen das Zusammenfügen von sich überschneidenden Daten, um fehlende Werte in einem Objekt mit Werten aus einem anderen zu ergänzen.
* Mit [pandas.merge_asof](https://pandas.pydata.org/docs/reference/api/pandas.merge_asof.html) könnt ihr zeitreihenbasierte *Window Joins* zwischen DataFrame-Objekten durchführen.

## Datenbankähnliche DataFrame-Joins

Merge- oder Join-Operationen kombinieren Datensätze durch die Verknüpfung von Zeilen mit einem oder mehreren Schlüsseln. Diese Operationen sind besonders wichtig in relationalen, SQL-basierten Datenbanken. Die Merge-Funktion in pandas ist der Haupteinstiegspunkt für die Anwendung dieser Algorithmen auf eure Daten.

In [1]:
import pandas as pd

In [2]:
encoding = pd.DataFrame({'Unicode': ['U+0000', 'U+0001', 'U+0002', 'U+0003', 'U+0004', 'U+0005'],
                         'Decimal': [0, 1, 2, 3, 4, 5],
                         'Octal': ['000', '001', '002', '003', '004', '005'],
                         'Key': ['NUL', 'Ctrl-A', 'Ctrl-B', 'Ctrl-C', 'Ctrl-D', 'Ctrl-E']})

update = pd.DataFrame({'Unicode': ['U+0003', 'U+0004', 'U+0005', 'U+0006', 'U+0007', 'U+0008', 'U+0009'],
                       'Decimal': [3, 4, 5, 6, 7, 8, 9],
                       'Octal': ['003', '004', '005', '006', '007', '008', '009'],
                       'Key': ['Ctrl-C', 'Ctrl-D', 'Ctrl-E', 'Ctrl-F', 'Ctrl-G', 'Ctrl-H', 'Ctrl-I']})

encoding, update

(  Unicode  Decimal Octal     Key
 0  U+0000        0   000     NUL
 1  U+0001        1   001  Ctrl-A
 2  U+0002        2   002  Ctrl-B
 3  U+0003        3   003  Ctrl-C
 4  U+0004        4   004  Ctrl-D
 5  U+0005        5   005  Ctrl-E,
   Unicode  Decimal Octal     Key
 0  U+0003        3   003  Ctrl-C
 1  U+0004        4   004  Ctrl-D
 2  U+0005        5   005  Ctrl-E
 3  U+0006        6   006  Ctrl-F
 4  U+0007        7   007  Ctrl-G
 5  U+0008        8   008  Ctrl-H
 6  U+0009        9   009  Ctrl-I)

Wenn wir `merge` mit diesen Objekten aufrufen, erhalten wir:

In [3]:
pd.merge(encoding, update)

,Unicode,Decimal,Octal,Key
0,U+0003,3,003,Ctrl-C
1,U+0004,4,004,Ctrl-D
2,U+0005,5,005,Ctrl-E


Standardmäßig führt `merge` einen sog. *Inner Join* durch; die Schlüssel im Ergebnis sind die Schnittmenge bzw. die gemeinsame Menge in beiden Tabellen.

<div class="alert alert-block alert-info">

**Hinweis:**

Ich habe nicht angegeben, über welche Spalte die Verknüpfung erfolgen soll. Wenn diese Information nicht angegeben wird, verwendet `merge` die sich überschneidenden Spaltennamen als Schlüssel. Es ist jedoch eine gute Praxis, dies explizit anzugeben:
</div>

In [4]:
pd.merge(encoding, update, on='Unicode')

,Unicode,Decimal_x,Octal_x,Key_x,Decimal_y,Octal_y,Key_y
0,U+0003,3,003,Ctrl-C,3,003,Ctrl-C
1,U+0004,4,004,Ctrl-D,4,004,Ctrl-D
2,U+0005,5,005,Ctrl-E,5,005,Ctrl-E


Wenn die Spaltennamen in jedem Objekt unterschiedlich sind, könnt ihr sie separat angeben. Im folgenden Beispiel erhält `update2` den Schlüssel `U+` und nicht `Unicode`:

In [5]:
update2 = pd.DataFrame({'U+': ['U+0003', 'U+0004', 'U+0005', 'U+0006', 'U+0007', 'U+0008', 'U+0009'],
                        'Decimal': [3, 4, 5, 6, 7, 8, 9],
                        'Octal': ['003', '004', '005', '006', '007', '008', '009'],
                        'Key': ['Ctrl-C', 'Ctrl-D', 'Ctrl-E', 'Ctrl-F', 'Ctrl-G', 'Ctrl-H', 'Ctrl-I']})

pd.merge(encoding, update2, left_on='Unicode', right_on='U+')

,Unicode,Decimal_x,Octal_x,Key_x,U+,Decimal_y,Octal_y,Key_y
0,U+0003,3,003,Ctrl-C,U+0003,3,003,Ctrl-C
1,U+0004,4,004,Ctrl-D,U+0004,4,004,Ctrl-D
2,U+0005,5,005,Ctrl-E,U+0005,5,005,Ctrl-E


Ihr könnt mit `merge` jedoch nicht nur einen *Inner Join* durchführen, mit dem die Schlüssel im Ergebnis die Schnittmenge bzw. die gemeinsame Menge in beiden Tabellen sind. Andere mögliche Optionen sind:

Option | Verhalten
:----- | :--------
`how='inner'` | verwendet nur die in beiden Tabellen beobachteten Schlüsselkombinationen
`how='left'` | verwendet alle in der linken Tabelle gefundenen Schlüsselkombinationen
`how='right'` | verwendet alle in der rechten Tabelle gefundenen Schlüsselkombinationen
`how='outer'` | verwendet alle in beiden Tabellen beobachteten Schlüsselkombinationen zusammen

In [6]:
pd.merge(encoding, update, on='Unicode', how='left')

,Unicode,Decimal_x,Octal_x,Key_x,Decimal_y,Octal_y,Key_y
0,U+0000,0,000,NUL,NaN,NaN,NaN
1,U+0001,1,001,Ctrl-A,NaN,NaN,NaN
2,U+0002,2,002,Ctrl-B,NaN,NaN,NaN
3,U+0003,3,003,Ctrl-C,3.0,003,Ctrl-C
4,U+0004,4,004,Ctrl-D,4.0,004,Ctrl-D
5,U+0005,5,005,Ctrl-E,5.0,005,Ctrl-E


In [7]:
pd.merge(encoding, update, on='Unicode', how='outer')

,Unicode,Decimal_x,Octal_x,Key_x,Decimal_y,Octal_y,Key_y
0,U+0000,0.0,000,NUL,NaN,NaN,NaN
1,U+0001,1.0,001,Ctrl-A,NaN,NaN,NaN
2,U+0002,2.0,002,Ctrl-B,NaN,NaN,NaN
3,U+0003,3.0,003,Ctrl-C,3.0,003,Ctrl-C
4,U+0004,4.0,004,Ctrl-D,4.0,004,Ctrl-D
5,U+0005,5.0,005,Ctrl-E,5.0,005,Ctrl-E
6,U+0006,NaN,NaN,NaN,6.0,006,Ctrl-F
7,U+0007,NaN,NaN,NaN,7.0,007,Ctrl-G
8,U+0008,NaN,NaN,NaN,8.0,008,Ctrl-H
9,U+0009,NaN,NaN,NaN,9.0,009,Ctrl-I


Die Join-Methode wirkt sich nur auf die eindeutigen Schlüsselwerte aus, die im Ergebnis erscheinen.

Um mehrere Schlüssel zusammenzuführen, könnt ihr eine Liste von Spaltennamen übergeben:

In [8]:
pd.merge(encoding, update, on=['Unicode', 'Decimal', 'Octal', 'Key'], how='outer')

,Unicode,Decimal,Octal,Key
0,U+0000,0,000,NUL
1,U+0001,1,001,Ctrl-A
2,U+0002,2,002,Ctrl-B
3,U+0003,3,003,Ctrl-C
4,U+0004,4,004,Ctrl-D
5,U+0005,5,005,Ctrl-E
6,U+0006,6,006,Ctrl-F
7,U+0007,7,007,Ctrl-G
8,U+0008,8,008,Ctrl-H
9,U+0009,9,009,Ctrl-I
